<a href="https://colab.research.google.com/github/tirtzash/HalachicAIInsight/blob/main/app3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tokenizers==0.10.2 transformers==4.6.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 17.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Norod78/hebrew-gpt_neo-small")
model = AutoModelForCausalLM.from_pretrained("Norod78/hebrew-gpt_neo-small", pad_token_id=tokenizer.eos_token_id)

prompt_text = "אני אוהב שוקולד ועוגות"
max_len = 512
sample_output_num = 3
seed = 1000

import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = 0 if torch.cuda.is_available()==False else torch.cuda.device_count()

print(f"device: {device}, n_gpu: {n_gpu}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.23M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/551M [00:00<?, ?B/s]

device: cpu, n_gpu: 0


In [3]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

try:
    test_text = "שלום!"
    inputs = tokenizer(test_text, return_tensors="pt", padding=True, truncation=True)
    print("Tokenization successful!")
    print(inputs)
except Exception as e:
    print(f"Error during tokenization: {e}")
# הגדרת פונקציה להפקת embeddings
def get_embedding2(text):
    # המרת הטקסט ל-input_ids
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # הפעלת המודל להפקת embeddings
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # קבלת hidden states של כל הטוקנים
    hidden_states = outputs.hidden_states[-1]  # בוחרים את השכבה האחרונה

    # חישוב הממוצע של ה-embeddings של כל הטוקנים
    embeddings = hidden_states.mean(dim=1).squeeze().numpy()

    return embeddings

# בדיקת הפונקציה עם טקסט לדוגמה
input_text = "שלום! מה שלומך היום?"
embedding = get_embedding2(input_text)
print(embedding)

Tokenization successful!
{'input_ids': tensor([[3347,    5]]), 'attention_mask': tensor([[1, 1]])}
[-5.31943655e+00 -2.39072108e+00 -4.87467557e-01  6.97738588e-01
  1.66282222e-01  2.08134794e+00 -1.26299584e+00  2.44471859e-02
  1.65020871e+00  1.33752656e+00  3.25201839e-01  5.28958321e-01
 -7.92511463e-01 -3.71504366e-01  4.33557719e-01 -2.23307163e-01
  5.21109402e-01  4.62652206e-01  4.69724864e-01  7.19825864e-01
 -3.16449106e-01  9.00686681e-01 -2.61785179e-01 -1.55111778e+00
 -2.32390928e+00  4.21775341e-01  2.47926450e+00  6.79237187e-01
 -1.02987790e+00 -3.81425284e-02 -1.15482795e+00  2.54340577e+00
 -4.23125267e+00  6.83022797e-01  1.72844127e-01  3.16697091e-01
 -1.23756276e-02 -1.72327232e+00  2.00868940e+00 -8.14793646e-01
  4.53600550e+00 -3.86855453e-01  2.03044939e+00  2.74076915e+00
  1.03219867e+00 -2.15183830e+00  1.35064274e-01  2.76538515e+00
  2.07499266e+00  2.06501153e-03 -5.14059998e-02 -2.49391341e+00
 -7.45885789e-01  2.14131689e+00  4.35733169e-01  7.9934

In [4]:
import numpy as np
import json
#from google.colab import drive
#drive.mount('/content/drive')
# קריאת הקובץ JSON
with open('sample_data/פרי גני שוע.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# שליפת כל השאלות מהקובץ
questions = []
for book in data['contain'].values():
    for chapter in book.values():
        for verse in chapter.values():
            for qa in verse:
                questions.append(qa['question'])

In [7]:
import numpy as np
import os
from google.colab import files
batch_size = 100
your_embedding_dimension = 768  # או כל מספר אחר בהתאם למודל שלך

def get_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

embeddings_filename = "sample_data/embeddings_step1.pkl"

# טען את הנתונים השמורים, אם קיימים
embeddings2 = load_from_pickle(embeddings_filename)
if embeddings2 is None:
    embeddings2 = np.array([]).reshape(0, your_embedding_dimension)  # וודא שהגדרת את your_embedding_dimension
else:
    print("נתונים שמורים נטענו בהצלחה.")

# קביעת נקודת התחלה לעיבוד
start_index = len(embeddings2)

for i, batch in enumerate(get_batches(questions, batch_size)):
    if i * batch_size < start_index:
        continue  # דלג על חלקים שכבר חושבו ושמורים


    # נניח שהפלט הוא כבר numpy.ndarray
    batch_embeddings = get_embedding2(batch)

   # שמור את החלק הנוכחי ל-embeddings2 ושמור לקובץ
    embeddings2 = np.vstack([embeddings2, batch_embeddings])
    save_to_pickle(embeddings2, embeddings_filename)


    # צייני את הנתיב לקובץ שברצונך להוריד
    files.download("sample_data/embeddings_step1.pkl")
    print(f"עיבוד ושמירת חלק {i+1} הסתיים בהצלחה.")




File sample_data/embeddings_step1.pkl not found!
Data saved to sample_data/embeddings_step1.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

עיבוד ושמירת חלק 1 הסתיים בהצלחה.
Data saved to sample_data/embeddings_step1.pkl


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

עיבוד ושמירת חלק 51 הסתיים בהצלחה.
Data saved to sample_data/embeddings_step1.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

עיבוד ושמירת חלק 52 הסתיים בהצלחה.


In [10]:
#import faiss
files.download("sample_data/embeddings_step1.pkl")
print(f"עיבוד ושמירת חלק {i+1} הסתיים בהצלחה.")
# # טען את ה-embeddings ששמרת
# embeddings2 = load_from_pickle("sample_data/embeddings_step1.pkl")

# # שלב נוסף - בניית FAISS index
# index_filename = "save_embded/index_step2.pkl"

# index2 = load_from_pickle(index_filename)
# if index2 is None:
#     # יצירת האינדקס אם הוא לא נטען מקובץ
#     dimension2 = embeddings2.shape[1]
#     index2 = faiss.IndexFlatL2(dimension2)
#     index2.add(embeddings2)
#     save_to_pickle(index2, index_filename)

# כעת יש לך את האינדקס מוכן לשימוש


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

עיבוד ושמירת חלק 52 הסתיים בהצלחה.


In [13]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 20.0 MB/s eta 0:00:00


In [15]:
import faiss

# טען את ה-embeddings ששמרת
embeddings2 = load_from_pickle("sample_data/embeddings_step1.pkl")

# שלב נוסף - בניית FAISS index
index_filename = "sample_data/index_step2.pkl"

index2 = load_from_pickle(index_filename)
if index2 is None:
    # יצירת האינדקס אם הוא לא נטען מקובץ
    dimension2 = embeddings2.shape[1]
    index2 = faiss.IndexFlatL2(dimension2)
    index2.add(embeddings2)
    save_to_pickle(index2, index_filename)

#כעת יש לך את האינדקס מוכן לשימוש

Data loaded from sample_data/embeddings_step1.pkl
File sample_data/index_step2.pkl not found!
Data saved to sample_data/index_step2.pkl


In [ ]:
import pickle

# שמירת ה-embeddings, השאלות והאינדקס
with open('save_embded/embeddings_and_index.pkl', 'wb') as f:
    pickle.dump({
        'embeddings': embeddings,
        'questions': questions,
        'faiss_index': index
    }, f)

print("ה-embeddings, השאלות והאינדקס נשמרו בהצלחה!")


In [5]:
import pickle
import os

def save_to_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Data saved to {filename}")

def load_from_pickle(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        print(f"Data loaded from {filename}")
        return data
    else:
        print(f"File {filename} not found!")
        return None



In [ ]:
# #טעינת קובץ ממקום קבוע
# import os
# import pickle

# # נתיב הקובץ בפרויקט שלך
# file_path = os.path.join(os.getcwd(), "save_embded/embeddings1.pkl")

# # בדיקה אם הקובץ קיים
# if os.path.exists(file_path):
#     with open(file_path, "rb") as f:
#         embeddings2 = pickle.load(f)
#     print("ה-embeddings נטענו בהצלחה!")
# else:
#     print(f"הקובץ לא נמצא בנתיב: {file_path}")


ה-embeddings נטענו בהצלחה!


In [17]:
# import numpy as np
# import faiss

# # הנחת שהנתונים הם כבר בפורמט numpy array
# # אם לא, תשתמש ב np.array כדי להמיר אותם
# embeddings2 = np.array(embeddings2).astype('float32')

# בדוק את הממדים של ה-embeddings
print("Dimensions of embeddings2:", embeddings2.shape)

dimension = embeddings2.shape[1]
# index2 = faiss.IndexFlatL2(dimension2)
# print("FAISS index:", index2)

# # הוספת הנתונים לאינדקס
# index2.add(embeddings2)

# פונקציה למציאת השאלות הקרובות ביותר
def find_nearest_questions2(query, k=5):
    query_embedding = get_embedding2(query).astype('float32').reshape(1, -1)
    print("Query embedding dimensions:", query_embedding.shape)

    # בדוק אם ממדי ה-query תואמים את ממדי האינדקס
    if query_embedding.shape[1] != dimension2:
        raise ValueError(f"Dimension mismatch: query embedding dimension {query_embedding.shape[1]} does not match index dimension {dimension}")

    distances, indices = index2.search(query_embedding, k)
    nearest_questions = [questions[idx] for idx in indices[0]]
    return nearest_questions

# שאלה לדוגמה
query = "האם נשים חיבות בתפילה"

# מציאת השאלות הקרובות ביותר
nearest_questions = find_nearest_questions2(query)
print("השאלות הקרובות ביותר לשאלה שלך הן:")
for q in nearest_questions:
    print(q)


Dimensions of embeddings2: (5158, 768)
Query embedding dimensions: (1, 768)
השאלות הקרובות ביותר לשאלה שלך הן:
כא. האם הבנות צריכות לכסות שערן בשעת הברכה והתפילה?
.2 גוף הפועל מצוי ומדוע אין יכול לשוכרו בהוזלה כמו בכל פסיקה ויש לו דמותר?
יג. אישה נשואה שרגילה ללכת בגילוי שיער מהו לברך כנגדה?
1. המחבק או המנשק אחת מן העריות שאין לבו נוקפו עליהם – להביא דוגמא והדין?
ד. האם מותר ללמד ילדים שדרכם לנגוע בנעליהם?
